In [3]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import random
import requests
import json

In [2]:
create_folder_path = os.path.dirname(os.path.realpath('__file__')) + '/Each_Company_Problem/'
if not os.path.exists(create_folder_path):
    os.makedirs(create_folder_path)
    print('Succesfully')
else:
    print('Path has already exists.')

Path has already exists.


In [3]:
User_ID = '1185437411@qq.com'
User_Password = '@Nhk960923'

In [4]:
proxy_arr = []
response = requests.get('https://acq.iemoapi.com/getProxyIp?regions=us&lb=1&return_type=json&protocol=http&num=40')
if response.status_code == 200:
    ip_file = json.loads(response.text)
ip_list = ip_file['data']
for ip in ip_list:
    proxy_arr.append('--proxy-server=http://' + str(ip['ip']) + ':' + str(ip['port']))

In [5]:
chrome_options = Options()
while True:
    ip_address = random.choice(proxy_arr)
    try:
        chrome_options.add_argument(ip_address)  
        driver = webdriver.Chrome(options=chrome_options)
        driver.get('https://leetcode.com/')
        time.sleep(3)
        break
    except:
        driver.close()
        print(ip_address)
        pass

In [16]:
def login():
    login_url = 'https://leetcode.com/accounts/login'
    driver.get(login_url)  # open login url
    # Explict wait
    # WebDriverWait(driver, 20).until_not(lambda item: item.find_element(By.CLASS_NAME, 'form__2-sN').is_displayed())  # is_display() return bool value
    try:
        driver.find_element(By.ID, 'id_login').send_keys(User_ID)
        driver.find_element(By.ID, 'id_password').send_keys(User_Password)
        driver.find_element(By.ID, 'signin_btn').click()     
    except:
        time.sleep(1)

In [38]:
driver = webdriver.Chrome()
login()

In [33]:
def Problems():
    # Initialization
    problem_df = pd.DataFrame()
    row_index = 0
    page = 1

    print('Start scrap problems info.')
    while page <= 52:
        frequency_index = 0
        problem_url = 'https://leetcode.com/problemset/all/?page=' + str(page)
        driver.get(problem_url)
        time.sleep(5)
    
        # click on button display label info
        driver.find_element(By.XPATH, '//button[@aria-label = "settings"]').click()
        driver.find_element(By.XPATH, '//div[@class = "rounded outline-none relative h-8 cursor-pointer select-none py-1.5 pl-2 pr-5 text-label-1 dark:text-dark-label-1 hover:bg-fill-4 dark:hover:bg-dark-fill-4"]').click()
        
        problems_info = driver.find_elements(By.XPATH, '//div[@role = "rowgroup"]//div[@role = "row"]')
        for index_1 in range(len(problems_info)):
            cell_tag = problems_info[index_1].find_elements(By.XPATH, 'div[@role = "cell"]')
            problem_df.loc[row_index, 'Title'] = cell_tag[1].text.split('\n')[0]
            problem_df.loc[row_index, 'Topic Label'] = str(cell_tag[1].text.split('\n')[1:])
            problem_df.loc[row_index, 'Accetpance'] = cell_tag[3].text
            problem_df.loc[row_index, 'Difficulty'] = cell_tag[4].text
            problem_df.loc[row_index, 'Frequency'] = cell_tag[5].find_elements(By.XPATH, '//div[@class = "absolute h-full rounded-full transition-all duration-300 ease-out bg-brand-orange dark:bg-dark-brand-orange"]')[frequency_index].get_dom_attribute('style').split(' ')[1]
            row_index += 1
            frequency_index += 1
        page += 1

    print('Page completed.')

    for index in range(len(problem_df)):
        problem_name = problem_df.loc[index, 'Title'].split('.')[1].replace(' ', '-').replace('(', '').replace(')', '')[1:]
        accecpt_url = 'https://leetcode.com/problems/' + problem_name
        driver.get(accecpt_url)
        time.sleep(3)
        try:
            problem_ac_sb = driver.find_elements(By.XPATH, '//div[@class = "mr-4 flex items-center space-x-2.5"]')
            problem_df.loc[index, 'Accepted'] = problem_ac_sb[0].text.split('\n')[1]
            problem_df.loc[index, 'Submissions'] = problem_ac_sb[1].text.split('\n')[1]
        except:
            print(index)
            pass

    return problem_df

In [ ]:
problems_info_df = Problems()

In [40]:
problems_info_df.head(5)

,Title,Topic Label,Accetpance,Difficulty,Frequency,Accepted,Submissions
0,208. Implement Trie (Prefix Tree),"['Hash Table', 'String', 'Design', '1+']",62.5%,Medium,26.1195%;,786.6K,1.3M
1,1. Two Sum,"['Array', 'Hash Table']",49.6%,Easy,99.4833%;,9.2M,18.5M
2,2. Add Two Numbers,"['Linked List', 'Math', 'Recursion']",40.3%,Medium,94.8402%;,3.5M,8.7M
3,3. Longest Substring Without Repeating Characters,"['Hash Table', 'String', 'Sliding Window']",33.8%,Medium,95.2438%;,4.3M,12.8M
4,4. Median of Two Sorted Arrays,"['Array', 'Binary Search', 'Divide and Conquer']",36.1%,Hard,92.7477%;,1.8M,5.1M


In [41]:
problems_info_df.to_csv(os.path.dirname(os.path.realpath('__file__')) + '/problems_info.csv', encoding = 'utf-8-sig')

In [10]:
def Problems_Premium():
    page = 1
    problem_df = pd.read_csv(os.path.dirname(os.path.realpath('__file__')) + '/problems_info.csv')
    
    # click the button
    driver.find_element(By.XPATH, '//button[@aria-label = "settings"]').click()
    driver.find_elements(By.XPATH, '//div[@class = "rounded outline-none relative h-8 cursor-pointer select-none py-1.5 pl-2 pr-5 text-label-1 dark:text-dark-label-1 hover:bg-fill-4 dark:hover:bg-dark-fill-4"]')[1].click()
    time.sleep(3)

    while page <= 11:
        try:
            nav_tag = driver.find_element(By.XPATH, '//nav[@class = "mb-6 md:mb-0 flex flex-nowrap items-center space-x-2"]')
            nav_tag.find_elements(By.TAG_NAME, 'button')[-1].click()
            time.sleep(3)
        except:
            pass

        problems_info = driver.find_elements(By.XPATH, '//div[@role = "rowgroup"]//div[@role = "row"]')
        for index_1 in range(len(problems_info)):
            cell_tag = problems_info[index_1].find_elements(By.XPATH, 'div[@role = "cell"]')
            problem_df.loc[problem_df['Title'] == cell_tag[1].text.split('\n')[0], ['Premium']] = 'Y' 
        page += 1

    return problem_df


In [89]:
problems_full_info_df = Problems_Premium()

In [95]:
problems_full_info_df.drop(problems_full_info_df.columns[[0,1]], axis = 1, inplace = True)
problems_full_info_df.to_csv(os.path.dirname(os.path.realpath('__file__')) + '/problems_info.csv', encoding = 'utf-8-sig')

In [153]:
def Companies():
    # Intitial
    Companies_df = pd.DataFrame()
    row_index = 0

    companies_url = 'https://leetcode.com/problemset/all/'
    driver.get(companies_url)
    time.sleep(2)

    button_tag = driver.find_element(By.XPATH, '//div[@class = "shadow-level1 dark:shadow-dark-level1 bg-layer-1 dark:bg-dark-layer-1 rounded-lg mt-4 px-4 pt-2 pb-1"]//button[@aria-label = "Next"]')
    while True:
        sw_class = driver.find_element(By.CLASS_NAME, 'swiper-wrapper')
        for company in sw_class.find_elements(By.XPATH, '//span[@data-tag = "tagslug"]'):
            temp = []
            if company.text != '':
                Companies_df.loc[row_index, 'Company'] = company.text.split('\n')[0]
                Companies_df.loc[row_index, 'Problem Number'] = company.text.split('\n')[1]
            row_index += 1
        # judge the button whether is_enabled
        if not button_tag.is_enabled():
            break
        button_tag.click()
        time.sleep(1)
        button_tag = driver.find_element(By.XPATH, '//div[@class = "shadow-level1 dark:shadow-dark-level1 bg-layer-1 dark:bg-dark-layer-1 rounded-lg mt-4 px-4 pt-2 pb-1"]//button[@aria-label = "Next"]')

    
    return Companies_df

In [136]:
companies_df = Companies()

In [131]:
companies_df

,Company,Problem Number
0,Amazon,1245
1,Google,1212
2,Microsoft,761
3,Facebook,691
4,Apple,641
...,...,...
3817,MachineZone,1
3818,CoderByte,1
3819,instagram,1
3820,Strava,1


In [132]:
companies_df.to_csv(os.path.dirname(os.path.realpath('__file__')) + '/companies_df.csv', encoding = 'utf-8-sig')

In [112]:
def Companies_Problem():
    company = pd.read_csv(os.path.dirname(os.path.realpath('__file__')) + '/companies_df.csv')

    for index in range(len(company)):
        # Initial 
        Each_company_problem = pd.DataFrame()
        row_index = 0

        company_name = company.loc[index, 'Company']
        # prevent 404 NOT FOUND
        if 'Walmart Global Tech' == company_name:
            company_name = 'walmart-labs'
        if ' ' in company_name:
            company_name = company_name.replace(' ', '-')
        if '.' in company_name:
            company_name = company_name.replace('.','')

        company_problem_url = 'https://leetcode.com/company/' + company_name
        driver.get(company_problem_url)
        time.sleep(1)
        tdbody_tag = driver.find_elements(By.XPATH, '//tbody[@class = "reactable-data"]//td[@label = "Title"]')
        for item in tdbody_tag:
            Each_company_problem.loc[row_index, 'Problem'] = item.text
            row_index += 1
        Each_company_problem.to_csv(os.path.dirname(os.path.realpath('__file__')) + '/Each_Company_Problem/' + company_name + '.csv')
        
    print('Completed.')

In [ ]:
Companies_Problem()

In [9]:
def Global_Ranking():
    # Intial
    Global_Ranking_df = pd.DataFrame()
    row_index = 0
    pages = 1

    global_ranking_url = 'https://leetcode.com/contest/globalranking/'
    driver.get(global_ranking_url)
    time.sleep(1)

    # Check max pages
    for page_btn in driver.find_elements(By.XPATH, '//ul[@class = "pagination"]//li[@class = "page-btn"]'):
        max_pages = int(page_btn.text.split('\n')[-1])

    while True:
        ranking_users = driver.find_elements(By.XPATH, '//div[@class = "top-users"]//div[@class = "ranking-row"]')
        for user in ranking_users:
            Global_Ranking_df.loc[row_index, 'Rank'] = user.text.split('\n')[0]
            Global_Ranking_df.loc[row_index, 'User Name'] = user.text.split('\n')[1]
            Global_Ranking_df.loc[row_index, 'Contests Attended'] = user.text.split('\n')[2].split(' ')[0]
            try:
                Global_Ranking_df.loc[row_index, 'Country'] = user.find_elements(By.TAG_NAME, 'span')[1].get_attribute('data-original-title')
            except:
                pass
            row_index += 1
        pages += 1
        global_ranking_url = 'https://leetcode.com/contest/globalranking/' + str(pages)
        if pages > int(max_pages/300):
            break
        driver.get(global_ranking_url)
        time.sleep(3)

    return Global_Ranking_df

In [17]:
global_ranking_df = Global_Ranking()

In [350]:
global_ranking_df

,Rank,User Name,Contests Attended,Country
0,1,neal_wu,51,United States
1,2,numb3r5,81,Australia
2,3,nyu_ldf,86,NaN
3,4,int65536,35,China
4,5,JOHNKRAM,220,China
...,...,...,...,...
1721,1721,xyseu1,31,China
1722,1722,eseo88,20,NaN
1723,1723,gqqggqqg,51,China
1724,1724,Attttrx,74,China


In [390]:
global_ranking_df.to_csv(os.path.dirname(os.path.realpath('__file__')) + '/global_ranking_df.csv', encoding = 'utf-8-sig')

In [6]:
def split_Country(df):
    Chinese_df = pd.DataFrame(columns = df.columns[1:])
    Other_df = pd.DataFrame(columns = df.columns[1:])
    Chinese_index = 0
    Other_index = 0

    for i in range(len(df)):
        if df.loc[i, 'Country'] == 'China' or (not str(df.loc[i, 'Country']).replace(' ', '').encode('utf-8').isalpha()):
            Chinese_df.loc[Chinese_index] = df.loc[i]
            Chinese_index += 1
        else:
            Other_df.loc[Other_index] = df.loc[i]
            Other_index += 1 

    return Chinese_df, Other_df        

In [7]:
def Ranking_User_Info(driver, proxy_arr):
    # Initial
    user_name_df = pd.read_csv(os.path.dirname(os.path.realpath('__file__')) + '/global_ranking_df.csv')
    Chinese_df, Other_df = split_Country(user_name_df)
    Ranking_User_Info_df = pd.DataFrame(columns = ['User Name', 'Languages', 'Advanced skill', 'Intermediate skill', 'Fundamental skill', '技能'])
    row_index = 0

    for Chinese_index in range(len(Chinese_df)):
        ranking_user_name = Chinese_df.loc[Chinese_index, 'User Name']
        # Check China or chinese words
        ranking_user_info_url = 'https://leetcode.cn/u/' + ranking_user_name
        driver.get(ranking_user_info_url)
        time.sleep(1)

        # Click show more hidden tag, China
        try:
            for show_more in driver.find_elements(By.XPATH, '//div[@class = "shadow-level3 dark:shadow-dark-level3 bg-layer-1 dark:bg-dark-layer-1 rounded-lg flex w-full min-w-[300px] flex-col px-4 pt-[21px] lc-lg:w-[300px]"]//span[@class = "cursor-pointer"]'):
                show_more.click()
        except:
            pass
        try:
            driver.find_elements(By.XPATH, '//div[@style = "margin: 7px 4px;"]')[-1].click()
        except:
            pass


        # Scrape Language
        languages = driver.find_elements(By.XPATH, '//span[@class = "inline-flex items-center px-2 whitespace-nowrap text-xs leading-6 rounded-full text-label-3 dark:text-dark-label-3 bg-fill-3 dark:bg-dark-fill-3"]')
        temp_language = []
        Ranking_User_Info_df.loc[row_index, 'User Name'] = ranking_user_name
        for language in languages:
            temp_language.append(language.text.split('\n')[0])
        Ranking_User_Info_df.loc[row_index, 'Languages'] = str(temp_language)
        # Scrape Skills
        try:
            skills = driver.find_elements(By.XPATH, '//div[@class = "my-4"]//div[@class = "flex flex-wrap flex justify-center"]')
            for skill in skills:
                Ranking_User_Info_df.loc[row_index, '技能'] = str(skill.text.split('\n')[:-1])
        except:
            pass
        if pd.isnull(Ranking_User_Info_df.loc[row_index, '技能']):
            Other_df.loc[len(Other_df)] = Chinese_df.loc[Chinese_index]

        row_index += 1

    # contorll times, sleep 300s so that ignore the defend by broswer
    step = 0

    for Other_index in range(len(Other_df)):
        ranking_user_name = Other_df.loc[Other_index, 'User Name']
        ranking_user_info_url = 'https://leetcode.com/' + ranking_user_name
        if step < 20:
            driver.get(ranking_user_info_url)
            time.sleep(3)
        else:
            driver.close()
            step = 0
            while True:
                proxy = random.choice(proxy_arr)
                chrome_options = Options()
                try:
                    chrome_options.add_argument(proxy)
                    driver = webdriver.Chrome(options=chrome_options)
                    driver.get(ranking_user_info_url)
                    time.sleep(5)
                    break
                except:
                    proxy_arr.remove(proxy)
                    driver.close()
                    print(proxy)
                    pass
            

        # Click show more hidden tag, Forigner
        try:
            for show_more in driver.find_elements(By.XPATH, '//div[@class = "shadow-level3 dark:shadow-dark-level3 bg-layer-1 dark:bg-dark-layer-1 rounded-lg flex w-full min-w-[300px] flex-col px-4 pt-[21px] lc-lg:w-[300px]"]//span[@class = "cursor-pointer"]'):
                show_more.click()
        except:
            pass          


        # Scrape Languages
        languages = driver.find_elements(By.XPATH, '//div[@class = "mt-4 flex flex-col space-y-3"]//div[@class = "flex items-center justify-between text-xs text-label-1 dark:text-dark-label-1"]')
        temp_language = []  # store temp language
        for language in languages:
            Ranking_User_Info_df.loc[row_index, 'User Name'] = ranking_user_name
            temp_language.append(language.text.split('\n')[0])
        Ranking_User_Info_df.loc[row_index, 'Languages'] = str(temp_language)
        # Scrape Skills
        try:
            skills = driver.find_elements(By.XPATH, '//div[@class = "mt-4 flex flex-col space-y-4"]')[1].find_elements(By.XPATH, '//div[@class = "mt-3 flex flex-wrap"]')
            Ranking_User_Info_df.loc[row_index, 'Advanced skill'] = str(skills[0].text.split('\n'))
            Ranking_User_Info_df.loc[row_index, 'Intermediate skill'] = str(skills[1].text.split('\n'))
            Ranking_User_Info_df.loc[row_index, 'Fundamental skill'] = str(skills[2].text.split('\n'))
        except:
            pass

        row_index += 1
        step += 1

    
    return Ranking_User_Info_df

In [8]:
ranking_user_Info_df = Ranking_User_Info(driver, proxy_arr)

In [9]:
ranking_user_Info_df

,User Name,Languages,Advanced skill,Intermediate skill,Fundamental skill,技能
0,int65536,['C++'],NaN,NaN,NaN,"['数组', '回溯', '二分查找', '位运算', '状态压缩', '计数', '动态规..."
1,JOHNKRAM,"['C++', 'JavaScript']",NaN,NaN,NaN,"['栈', '贪心', '位运算', '树', '深度优先搜索', '广度优先搜索', '并..."
2,endlesscheng,[],NaN,NaN,NaN,[]
3,arignote,[],NaN,NaN,NaN,[]
4,moransky,[],NaN,NaN,NaN,[]
...,...,...,...,...,...,...
1775,immike,"['Python3', 'Python', 'C++', 'Go']","['Dynamic Programming', 'x88', 'Backtracking',...","['Hash Table', 'x130', 'Math', 'x107', 'Greedy...","['Array', 'x343', 'String', 'x157', 'Sorting',...",NaN
1776,jiaxiangwu,"['C++', 'Python3', 'Java', 'Rust']","['Dynamic Programming', 'x105', 'Backtracking'...","['Hash Table', 'x136', 'Math', 'x136', 'Greedy...","['Array', 'x390', 'String', 'x162', 'Sorting',...",NaN
1777,tyhaha,"['Python3', 'Python']","['Dynamic Programming', 'x24', 'Backtracking',...","['Greedy', 'x29', 'Math', 'x28', 'Hash Table',...","['Array', 'x95', 'String', 'x39', 'Sorting', '...",NaN
1778,qb_2008,"['C++', 'Rust', 'Kotlin', 'Java', 'MySQL', 'Ja...","['Dynamic Programming', 'x158', 'Backtracking'...","['Math', 'x154', 'Hash Table', 'x153', 'Depth-...","['Array', 'x403', 'String', 'x224', 'Sorting',...",NaN


In [11]:
ranking_user_Info_df.to_csv(os.path.dirname(os.path.realpath('__file__')) + '/ranking_user_Info_df.csv', encoding = 'utf-8-sig')

In [ ]:
driver.close()